<a href="https://colab.research.google.com/github/WESTZERO115/childmeal/blob/main/foodnutrtion_to_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql

     |████████████████████████████████| 43 kB 974 kB/s 


In [ ]:
import json
import requests
import pymysql


myKey = "95ec7adda481472f8165"
dataType = "json"
startNum = "90001"
endNum = "90608"

url = 'http://openapi.foodsafetykorea.go.kr/api/'+myKey+'/I2790/'+dataType+'/'+startNum+'/'+endNum  #식품영양정보DB/공공API/keyvalue/DBID/type/시작value/끝value 

res = requests.get(url)
res.encoding = 'UTF-8'
res_message = res.text

print(res_message)
data = json.loads(res_message)

## print(json.dumps(data, ensure_ascii=False, indent=2)) # ensure_ascii=False --> json 파일 호출시 한글 깨짐 방지

foods = data["I2790"]["row"]
food_list=[]
i=1
for food in foods:
    food_code = food["FOOD_CD"]
    food_menu = food["DESC_KOR"]
    food_groupname = food["GROUP_NAME"]
    if food_groupname == '':
        food_groupname = '식품군 정보없음'
    food_servingsize = food["SERVING_SIZE"]
    if food_servingsize == '':
        food_servingsize = '총 내용량 정보없음'
    food_kcal = food["NUTR_CONT1"]
    if food_kcal == '':
        food_kcal = '열량(kcal)(1회제공량당) 정보없음'
    food_carbohyd = food["NUTR_CONT2"]
    if food_carbohyd == '':
        food_carbohyd = '탄수화물(g)(1회제공량당) 정보없음'
    food_protein = food["NUTR_CONT3"]
    if food_protein == '':
        food_protein = '단백질(g)(1회제공량당) 정보없음'
    food_fat = food["NUTR_CONT4"]
    if food_fat == '':
        food_fat = '지방(g)(1회제공량당) 정보없음'
    food_sugars = food["NUTR_CONT5"]
    if food_sugars == '':
        food_sugars = '당류(g)(1회제공량당) 정보없음'
    food_natrium = food["NUTR_CONT6"]
    if food_natrium == '':
        food_natrium = '나트륨(mg)(1회제공량당) 정보없음'
    food_cholesterol = food["NUTR_CONT7"]
    if food_cholesterol == '':
        food_cholesterol = '콜레스테롤(mg)(1회제공량당) 정보없음'
    food_saturatedfatty = food["NUTR_CONT8"]
    if food_saturatedfatty == '':
        food_saturatedfatty = '포화지방산(g)(1회제공량당) 정보없음'
    food_transfat = food["NUTR_CONT9"]
    if food_transfat == '':
        food_transfat = '트랜스지방(g)(1회제공량당) 정보없음'

    food_list.append(tuple([food_code, food_menu, food_groupname, food_servingsize, food_kcal, food_carbohyd, food_protein, food_fat, food_sugars, food_natrium, food_cholesterol, food_saturatedfatty, food_transfat]))
    i+=1
    print(i)
print(food_list)

import pandas as pd

df = pd.DataFrame(food_list, columns = ['food_code', 'food_menu', 'food_groupname','food_servingsize','food_kcal','food_carbohyd','food_protein','food_fat','food_sugars','food_natrium','food_cholesterol','food_saturatedfatty','food_transfat'])
df


In [ ]:
import pymysql

#테이블 생성
conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
sql= "CREATE TABLE FOOD_NUTRITION(food_code VARCHAR(7),food_menu VARCHAR(10),food_groupname VARCHAR(10),food_servingsize INT,food_kcal FLOAT(5),food_carbohyd FLOAT(5),food_protein FLOAT(5),food_fat FLOAT(5),food_sugars FLOAT(5),food_natrium FLOAT(8),food_cholesterol FLOAT(6),food_saturatedfatty FLOAT(5),food_transfat FLOAT(3));" #DB생성
cursor.execute(sql) #SQL수행
conn.commit()
conn.close()


In [66]:
def mysql_save(food_list):
    conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
    cursor=conn.cursor()
    
    # 데이터 삽입
    sql = "INSERT INTO FOOD_NUTRITION(food_code,food_menu,food_groupname,food_servingsize,food_kcal,food_carbohyd,food_protein,food_fat,food_sugars,food_natrium,food_cholesterol,food_saturatedfatty,food_transfat) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    cursor.executemany(sql,food_list)
    conn.commit()
    conn.close()
mysql_save(food_list)

In [5]:
conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
cursor.execute("SELECT * FROM FOOD_NUTRITION WHERE food_menu = '우리쌀 누룽지';") 
(food_code,food_menu,food_groupname,food_servingsize,food_kcal,food_carbohyd,food_protein,food_fat,food_sugars,food_natrium,food_cholesterol,food_saturatedfatty,food_transfat) = cursor.fetchone() 
print(f"{food_code},{food_menu},{food_groupname},{food_servingsize},{food_kcal},{food_carbohyd},{food_protein},{food_fat},{food_sugars},{food_natrium},{food_cholesterol},{food_saturatedfatty},{food_transfat}")

P000426,우리쌀 누룽지,과자류, 빵류 또는,30,114.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
